In [25]:
from pyquil import Program,get_qc
from pyquil.api import WavefunctionSimulator
import sympy as smp
import numpy as np

class ProgramOutput:
    def __init__(self,P=""):
        self.qc = get_qc("9q-square-qvm")
        self.wfs = WavefunctionSimulator()
        P = "\n".join([f"DECLARE s{i} BIT[32]" for i in range(32)]) + P
        P = Program(P)
        P = self.qc.compile(P)
        P = str(P)
        P = P.split("\n")
        self.Program = P
        self.end_line = len(P)
        self.Program = [f"I {i}" for i in range(9)] + self.Program
        #self.Program = [f"DECLARE s{i} BIT[32]" for i in range(32)] + self.Program

    def run(self,end_line=None):
        if end_line is None: end_line = len(self.Program)-41
        P = self.Program[:41 + end_line]
        P = "\n".join(P)
        P = Program(P)
        self.outqc = self.qc.run(P).get_register_map()
        self.outwfs = self.wfs.wavefunction(P)
        self.end_line = end_line
    
    def step(self):
        self.end_line = min(self.end_line + 1,len(self.Program)-41)
        self.run(self.end_line)

    def __repr__(self):
        s = [""]
        state = str(self.outwfs)
        if len(state) < 100:
            s.append("State :\t" + state)    
        else:
            s.append("State :\t" + state[:30] + " .... " + state[-30:])
        psi = [self.outwfs[i] for i in range(2**9)]
        s.append("Psi :\t " + str(psi))
        # Registers
        s.append("\nRegisters\n")
        for i in range(32):
            reg = f"s{i}"
            regval = self.outqc[reg][0]
            regval_bin = "".join([str(x) for x in regval])
            regval_int = sum([regval[i]*2**(31-i) for i in range(32)])
            if regval_int > 2**32 : regval_int = regval_int - 2**32
            regval_int = str(regval_int)
            reg_string = reg + "\t : 0b" +  regval_bin + " = " + regval_int
            s.append(reg_string)
        return "\n".join(s)

    def display(self,states=None,regs=range(32),canvas=None,ax=None):
        psi = [str(np.round(self.outwfs[i],4)) for i in range(2**9)]
        if states is None:
            psi = [psi[i] + r"& |" + str(i) + r"\rangle" for i in range(2**9) if psi[i]!="0j"]
            if len(psi) > 32:
                psi = psi[:15] + [r"\vdots"]*2 +  psi[-15:] 
        else:
            psi = [psi[i] + r"& |" + str(i) + r"\rangle" for i in states] + [r"\vdots"]
        psi = r"\begin{bmatrix}" + (r" \\ ").join(psi) + r"\end{bmatrix}"
        #psi = r"\begin{matrix} +"+r"\rangle \\".join(str(self.outwfs).split(">"))+r"\end{matrix}"
        s = [r"\text{register} & \text{binary value} & \text{signed decimal}"]
        for i in regs:
            reg = f"s{i}"
            regval = self.outqc[reg][0]
            regval_bin = "".join([str(x) for x in regval])
            regval_int = sum([regval[i]*2**(31-i) for i in range(32)])
            if regval_int > 2**32 : regval_int = regval_int - 2**32
            regval_int = str(regval_int)
            reg_string = reg + " & " +  regval_bin + " & " + regval_int
            s.append(reg_string)
        R = r"\begin{bmatrix}" + r"\\ ".join(s) +r" \end{bmatrix}"
        P = self.Program[41:]
        ran = P[:self.end_line]
        not_ran = P[self.end_line:]
        ran = [r"\underline{\text{" + x + r"}}" for x in ran]
        not_ran = [r"\text{" + x + r"}" for x in not_ran]
        P = ran + not_ran
        P = r"\\ ".join(P)
        P = r"\begin{bmatrix}" + P + r"\end{bmatrix}"
        Full = r"$$ \boxed{\begin{matrix}" + r"P & |\Psi\rangle & C \\" + r" &".join([P,psi,R])+ r"\end{matrix}} $$"
        smp.preview(Full,viewer="file",filename="QVMout.png")

    def run_and_display(self,i=None):
        self.run(i)
        self.display()
        #display_latex(Latex(self.display()))

    #def interact(self):
    #    return interactive(self.run_and_display,{"manual":True},i=IntSlider(min=0, max=len(self.Program)-41, step=1))



In [26]:
P = open("QuilProgram.txt",'r').read()
PO = ProgramOutput(P)
PO.run_and_display()
PO.Program

['I 0',
 'I 1',
 'I 2',
 'I 3',
 'I 4',
 'I 5',
 'I 6',
 'I 7',
 'I 8',
 'DECLARE s0 BIT[32]',
 'DECLARE s1 BIT[32]',
 'DECLARE s2 BIT[32]',
 'DECLARE s3 BIT[32]',
 'DECLARE s4 BIT[32]',
 'DECLARE s5 BIT[32]',
 'DECLARE s6 BIT[32]',
 'DECLARE s7 BIT[32]',
 'DECLARE s8 BIT[32]',
 'DECLARE s9 BIT[32]',
 'DECLARE s10 BIT[32]',
 'DECLARE s11 BIT[32]',
 'DECLARE s12 BIT[32]',
 'DECLARE s13 BIT[32]',
 'DECLARE s14 BIT[32]',
 'DECLARE s15 BIT[32]',
 'DECLARE s16 BIT[32]',
 'DECLARE s17 BIT[32]',
 'DECLARE s18 BIT[32]',
 'DECLARE s19 BIT[32]',
 'DECLARE s20 BIT[32]',
 'DECLARE s21 BIT[32]',
 'DECLARE s22 BIT[32]',
 'DECLARE s23 BIT[32]',
 'DECLARE s24 BIT[32]',
 'DECLARE s25 BIT[32]',
 'DECLARE s26 BIT[32]',
 'DECLARE s27 BIT[32]',
 'DECLARE s28 BIT[32]',
 'DECLARE s29 BIT[32]',
 'DECLARE s30 BIT[32]',
 'DECLARE s31 BIT[32]',
 'RZ(pi) 2',
 'RX(pi/2) 2',
 'RZ(pi/2) 2',
 'RX(-pi/2) 2',
 'RZ(pi) 0',
 'RX(pi/2) 0',
 'RZ(pi/2) 0',
 'RX(-pi/2) 0',
 'RZ(pi) 1',
 'RX(pi/2) 1',
 'RZ(pi/2) 1',
 'RX(-pi/

In [27]:
import tkinter as tk
from tkinter import PhotoImage,Button

# Create the main window
parent = tk.Tk()
parent.title("Image in Tkinter")

# Load the image 
image = PhotoImage(file="QVMout.png")

# Create a label to display the image
image_label = tk.Label(parent, image=image)
def compute():
    P = open("QuilProgram.txt",'r').read()
    print(P)
    PO = ProgramOutput(P)
    PO.run_and_display()
    image = PhotoImage(file="QVMout.png")
    image_label.configure(image=image)
    image_label.image = image
B=Button(command = compute)
B.pack()
image_label.pack()

# Start the Tkinter event loop
parent.mainloop()


H 0
H 1
H 2
X 3
H 0
H 1
H 2

H 0
H 0
RX(-pi/2) 0
H 0
CNOT 1


Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/tmp/ipykernel_141081/665356847.py", line 16, in compute
    PO = ProgramOutput(P)
  File "/tmp/ipykernel_141081/895613356.py", line 12, in __init__
    P = self.qc.compile(P)
  File "/home/hp/.local/lib/python3.10/site-packages/pyquil/api/_quantum_computer.py", line 394, in compile
    nq_program = self.compiler.quil_to_native_quil(program, protoquil=protoquil)
  File "/home/hp/.local/lib/python3.10/site-packages/pyquil/api/_abstract_compiler.py", line 126, in quil_to_native_quil
    result = self._compile_with_quilc(
  File "/home/hp/.local/lib/python3.10/site-packages/pyquil/api/_abstract_compiler.py", line 145, in _compile_with_quilc
    return compile_program(
quilc.QuilcError: Problem compiling quil program: compilation error from RPCQ: Received error message from server: Unhandled error in host program:
Erro

H 0
X 0
H 0
X 0
X 1
H 0
X 0
X 1
MEASURE 0 s0[1]
H 0
X 0
X 1
MEASURE 0 s0[31]
H 0
X 0
X 1
MEASURE 0 s0[31]
H 0
X 0
X 1
MEASURE 0 s0[31]
